In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import ignite
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
import string
import math
from torch.autograd import Variable
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)
np.random.seed(123)
torch.manual_seed(123)
#https://www.analyticsvidhya.com/blog/2019/01/guide-pytorch-neural-networks-case-studies/
from sklearn.metrics import accuracy_score
from sklearn import linear_model, metrics, model_selection, ensemble


Using TensorFlow backend.


In [115]:
basic = pd.read_csv('./data/PBP - 2016 - Week 1.csv')
df3 = pd.read_csv('./data/PBP - 2016 - Week 3.csv')
df4 = pd.read_csv('./data/PBP - 2016 - Week 4.csv')
df5 = pd.read_csv('./data/PBP - 2016 - Week 5.csv')
df6 = pd.read_csv('./data/PBP - 2016 - Week 6.csv')
df7 = pd.read_csv('./data/PBP - 2016 - Week 7.csv')
df8 = pd.read_csv('./data/PBP - 2016 - Week 8.csv')
basic = pd.concat([basic, df3, df4, df5, df6, df7, df8])
offense_stats = pd.read_csv('./data/Offense Stats 2016.csv')

In [116]:
offense_stats

,Rk,School,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
0,1,Western Kentucky,14,45.5,21.8,33.6,64.8,336.8,3.0,34.8,186.4,5.4,2.5,68.4,523.1,7.6,13.1,9.9,6.6,66.7
1,2,Louisiana Tech,14,44.3,25.9,38.8,66.7,363.4,3.1,29.6,151.6,5.1,2.1,68.4,514.9,7.5,15.0,8.5,5.1,48.2
2,3,Oklahoma,13,43.9,20.8,29.5,70.6,318.0,3.3,44.0,236.8,5.4,2.3,73.5,554.8,7.5,13.5,10.5,5.1,52.2
3,4,South Florida,13,43.8,17.3,27.5,62.8,226.2,2.0,43.6,285.8,6.6,3.6,71.2,512.0,7.2,9.3,12.6,6.4,58.7
4,5,Texas Tech,12,43.7,35.7,54.4,65.5,463.0,3.9,32.4,103.6,3.2,1.8,86.8,566.6,6.5,19.8,7.7,7.8,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,124,Texas State,12,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3
124,125,Fresno State,12,17.7,17.5,33.8,51.9,213.1,1.1,36.7,116.2,3.2,0.9,70.4,329.3,4.7,8.5,6.9,6.3,55.8
125,126,Buffalo,12,16.5,17.7,33.1,53.4,200.5,0.9,35.5,153.3,4.3,1.0,68.6,353.8,5.2,9.3,8.0,5.3,43.1
126,127,Rutgers,12,15.7,13.3,27.7,47.9,138.3,0.9,40.7,145.3,3.6,0.7,68.3,283.5,4.1,6.8,8.0,5.3,43.0


In [117]:
basic['homeTeam'].unique()

array(['Baylor', 'Iowa State', 'Temple', 'Texas', 'Florida', 'Vanderbilt',
       'Alabama', 'Arkansas', 'Auburn', 'UNC', 'Kentucky', 'Wisconsin',
       'Miss St', 'West Virginia', 'Tennessee', 'Texas A&M', 'FSU', 'Cal',
       'Utah', 'Minnesota', 'Colorado', 'Stanford', 'Arizona State',
       'Oregon', 'Washington', 'Washington St', 'Utah State', 'UNLV',
       'Air Force', 'Louisiana', 'New Mexico', 'Wyoming', 'Nebraska',
       'Michigan', 'Nevada', 'Tulsa', 'Cent Michigan', 'Georgia State',
       'Buffalo', 'OSU', 'Penn State', 'Iowa', 'E Michigan', 'Ohio',
       'Arkansas State', 'Akron', 'Northwestern', 'Louisville', 'FIU',
       'FAU', 'Mid Tennessee', 'North Texas', 'Old Dominion',
       'W Kentucky', 'UTEP', 'UTSA', 'Wake Forest', 'NC State',
       'Syracuse', 'Boston College', 'Duke', 'Pitt', 'UVA', 'VT',
       'Cincinnati', 'UConn', 'UCF', 'ECU', 'USF', 'Houston', 'Memphis',
       'Navy', 'Purdue', 'Kansas', 'Oklahoma State', 'TCU', 'Texas Tech',
       'Michigan S

In [118]:
df = basic.join(offense_stats.set_index('School'), on='homeTeam')

In [119]:
df

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
0,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,15:00,2016-09-03T06:36:41Z,Kickoff,1,10,35,10,86,"Drew Galitz kickoff for 61 yds , Shakeir Ryan ...",35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
1,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,1,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Timeout,1,10,86,0,86,"Timeout NORTHWESTERN ST, clock 14:55",35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
2,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,2,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,1,10,86,-1,87,De'Mard Llorens run for a loss of 1 yard to th...,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
3,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,3,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,2,11,87,8,79,De'Mard Llorens run for 8 yds to the NWSt 21,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
4,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,4,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,3,3,79,0,79,De'Mard Llorens run for no gain to the NWSt 21,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10285,400869834,2016,8,326,Texas State,TXST,309,Louisiana,ULL,21,7,309,Louisiana,ULL,326,Texas State,TXST,3,27,4,6:27,2016-10-23T08:53:13Z,Penalty,1,15,60,-5,65,"UL LAFAYETTE Penalty, false start (-5 Yards) t...",124.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3
10286,400869834,2016,8,326,Texas State,TXST,309,Louisiana,ULL,21,8,309,Louisiana,ULL,326,Texas State,TXST,3,27,4,6:27,2016-10-23T08:53:13Z,Rush,1,20,65,19,46,Jordan Davis run for 19 yds to the TexSt 46,124.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3
10287,400869834,2016,8,326,Texas State,TXST,309,Louisiana,ULL,21,9,309,Louisiana,ULL,326,Texas State,TXST,3,27,4,6:27,2016-10-23T08:53:13Z,Rush,2,1,46,3,43,Raymond Calais run for 3 yds to the TexSt 43 f...,124.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3
10288,400869834,2016,8,326,Texas State,TXST,309,Louisiana,ULL,21,10,309,Louisiana,ULL,326,Texas State,TXST,3,27,4,6:27,2016-10-23T08:53:13Z,Rush,1,10,43,4,39,Raymond Calais run for 4 yds to the TexSt 39,124.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3


In [120]:
df = df.dropna()

In [121]:
df = df.replace('Pass Incompletion', 'Pass')
df = df.replace('Pass Reception', 'Pass')
df = df.replace('Passing Touchdown', 'Pass')
df = df.replace('Rushing Touchdown', 'Rush')
df = df.replace('Sack', 'Pass')

In [122]:
offense = df.query('homeTeam == offenseTeam')
offense = offense[(offense.type == 'Pass') | (offense.type == 'Rush')]

In [123]:
offense

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
6,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,0,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Pass,1,10,41,6,47,Seth Russell pass complete to KD Cannon for 6 ...,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
7,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Rush,2,4,47,3,50,Shock Linwood run for 3 yds to the 50 yard line,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
9,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,3,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Rush,3,11,40,42,82,Shock Linwood run for 42 yds to the NWSt 18 fo...,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
10,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,4,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Pass,1,10,82,5,87,Seth Russell pass complete to Quan Jones for 5...,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
11,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,5,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Pass,2,5,87,0,87,Seth Russell pass incomplete,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10265,400869834,2016,8,326,Texas State,TXST,309,Louisiana,ULL,18,13,326,Texas State,TXST,309,Louisiana,ULL,3,20,4,15:00,2016-10-23T08:34:46Z,Rush,3,7,58,1,59,Tyler Jones run for 1 yd to the LaLaf 41,124.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3
10267,400869834,2016,8,326,Texas State,TXST,309,Louisiana,ULL,18,15,326,Texas State,TXST,309,Louisiana,ULL,3,20,4,15:00,2016-10-23T08:34:46Z,Pass,4,11,54,0,54,"Tyler Jones pass incomplete to Mason Hays, bro...",124.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3
10274,400869834,2016,8,326,Texas State,TXST,309,Louisiana,ULL,20,1,326,Texas State,TXST,309,Louisiana,ULL,3,27,4,8:11,2016-10-23T08:51:28Z,Pass,1,10,22,4,26,Tyler Jones pass complete to Brendon Rushing f...,124.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3
10275,400869834,2016,8,326,Texas State,TXST,309,Louisiana,ULL,20,2,326,Texas State,TXST,309,Louisiana,ULL,3,27,4,8:11,2016-10-23T08:51:28Z,Rush,2,6,26,1,27,Stedman Mayberry run for 1 yd to the TexSt 27,124.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5,6.8,57.3


In [124]:
def invert_ranking(df):
    """This is so the higher the ranking the better"""
    max_rank = df['Rk'].max()
    for index,row in df.iterrows():
        df.at[index,'Rk'] = max_rank - int(row['Rk'])
    return df

In [125]:
offense = invert_ranking(offense)

In [126]:
offense.columns

Index(['gameId', 'year', 'week', 'homeId', 'homeTeam', 'homeAbbr', 'awayId',
       'awayTeam', 'awayAbbr', 'driveIndex', 'playIndex', 'offenseId',
       'offenseTeam', 'offenseAbbr', 'defenseId', 'defenseTeam', 'defenseAbbr',
       'homeScore', 'awayScore', 'quarter', 'clock', 'wallclock', 'type',
       'down', 'distance', 'yardLine', 'yardsGained', 'endYardLine',
       'description', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps',
       'completion percentage', 'passing yards', 'Passing Touchdowns',
       'Rush Attempts', 'Rushing yards', 'rush average', 'Rush Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush', 'Number of Penalties',
       'Penalty yards'],
      dtype='object')

In [127]:
result = offense[['yardsGained']]

In [128]:
result

,yardsGained
6,6
7,3
9,42
10,5
11,0
...,...
10265,1
10267,0
10274,4
10275,1


In [129]:
features = offense[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps',
       'completion percentage', 'passing yards', 'Passing Touchdowns',
       'Rush Attempts', 'Rushing yards', 'rush average', 'Rush Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]

In [130]:
features

,down,distance,yardLine,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush
6,1,10,41,68.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
7,2,4,47,68.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
9,3,11,40,68.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
10,1,10,82,68.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
11,2,5,87,68.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10265,3,7,58,3.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5
10267,4,11,54,3.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5
10274,1,10,22,3.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5
10275,2,6,26,3.0,12.0,18.6,21.4,34.6,61.9,219.9,1.2,35.5,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5


In [131]:
features = features.drop(columns=['Total yards', 'Pass Attemps', 'Rush Attempts', 'Rush Touchdowns', 'Passing Touchdowns', 'First down by pass', 'First down by rush'])

In [132]:
from sklearn.linear_model import LinearRegression


##  Making a rushing model

In [133]:
rushing = offense.query('type == "Rush"')

In [134]:
rushing_result = rushing['yardsGained']

In [135]:
rushing_features = rushing[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts',
                            'Rushing yards', 'rush average', 'Rush Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]

In [136]:
rushing_features

,down,distance,yardLine,Rk,G,Pts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush
7,2,4,47,68.0,13.0,34.6,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
9,3,11,40,68.0,13.0,34.6,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
12,3,5,87,68.0,13.0,34.6,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
20,2,2,77,23.0,13.0,34.6,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
21,1,7,93,23.0,13.0,34.6,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10260,3,8,45,3.0,12.0,18.6,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5
10261,4,1,52,3.0,12.0,18.6,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5
10262,1,10,55,3.0,12.0,18.6,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5
10265,3,7,58,3.0,12.0,18.6,82.4,2.3,1.1,70.1,302.3,4.3,10.8,5.5


In [137]:
rushing_result.describe()

count    12714.000000
mean         5.371874
std          8.316924
min        -22.000000
25%          1.000000
50%          3.000000
75%          7.000000
max         92.000000
Name: yardsGained, dtype: float64

In [138]:
rush_reg = LinearRegression().fit(rushing_features, rushing_result)

In [139]:
y_pred = rush_reg.predict(rushing_features)


math.sqrt(metrics.mean_squared_error(rushing_result, y_pred))

8.23191394576952

In [140]:
pd.DataFrame(y_pred).describe()

,0
count,12714.000000
mean,5.371874
std,1.183844
min,1.240299
25%,4.571798
50%,5.404436
75%,6.200116
max,9.726275


In [141]:
rushing_result.head()

7      3
9     42
12    13
20    16
21     5
Name: yardsGained, dtype: int64

## Passing Model

In [142]:
passing = offense.query('type == "Pass"')

In [143]:
passing_result = passing['yardsGained']

In [144]:
passing_features = passing[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps',
       'completion percentage', 'passing yards', 'Passing Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]

In [145]:
pass_reg = LinearRegression().fit(passing_features, passing_result)

In [146]:
pass_pred = pass_reg.predict(passing_features)


math.sqrt(metrics.mean_squared_error(passing_result, pass_pred))

11.697670244366087

In [147]:
passing_result.describe()

count    10312.000000
mean         7.168542
std         11.817064
min        -22.000000
25%          0.000000
50%          4.000000
75%         11.000000
max         92.000000
Name: yardsGained, dtype: float64

In [148]:
pd.DataFrame(pass_pred).describe()

,0
count,10312.000000
mean,7.168542
std,1.671601
min,1.478451
25%,6.013799
50%,7.186365
75%,8.339774
max,12.229464


In [149]:
def rush_feature_label_split(df):
    rushing = df.query('type == "Rush"')
    rushing_result = rushing['yardsGained']
    rushing_features = rushing[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts',
                            'Rushing yards', 'rush average', 'Rush Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]
    return rushing_features, rushing_result

In [150]:
def pass_feature_label_split(df):
    passing = df.query('type == "Pass"')
    passing_result = passing['yardsGained']
    passing_features = passing[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps',
       'completion percentage', 'passing yards', 'Passing Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]
    return passing_features, passing_result

## Testing the models with week 2 data

In [151]:
week2 = pd.read_csv('./data/PBP - 2016 - Week 2.csv')

In [152]:
week2 = week2.join(offense_stats.set_index('School'), on='homeTeam')

In [153]:
week2 = week2.replace('Pass Incompletion', 'Pass')
week2 = week2.replace('Pass Reception', 'Pass')
week2 = week2.replace('Passing Touchdown', 'Pass')
week2 = week2.replace('Rushing Touchdown', 'Rush')
week2 = week2.replace('Sack', 'Pass')
week2 = week2.query('homeTeam == offenseTeam')
week2 = week2[(week2.type == 'Pass') | (week2.type == 'Rush')]

In [154]:
week2 = week2.dropna()

In [155]:
week2

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
6,400868877,2016,2,239,Baylor,BAY,2567,SMU,SMU,1,0,239,Baylor,BAY,2567,SMU,SMU,0,0,1,14:02,2016-09-11T02:43:41Z,Pass,1,10,21,0,21,Seth Russell pass incomplete,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
7,400868877,2016,2,239,Baylor,BAY,2567,SMU,SMU,1,1,239,Baylor,BAY,2567,SMU,SMU,0,0,1,14:02,2016-09-11T02:43:41Z,Pass,2,10,21,8,29,Seth Russell pass complete to Chris Platt for ...,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
8,400868877,2016,2,239,Baylor,BAY,2567,SMU,SMU,1,2,239,Baylor,BAY,2567,SMU,SMU,0,0,1,14:02,2016-09-11T02:43:41Z,Rush,3,2,29,8,37,Seth Russell run for 8 yds to the Bayl 37 for ...,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
9,400868877,2016,2,239,Baylor,BAY,2567,SMU,SMU,1,3,239,Baylor,BAY,2567,SMU,SMU,0,0,1,14:02,2016-09-11T02:43:41Z,Rush,1,10,37,-4,33,Lynx Hawthorne run for a loss of 4 yards to th...,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
10,400868877,2016,2,239,Baylor,BAY,2567,SMU,SMU,1,4,239,Baylor,BAY,2567,SMU,SMU,0,0,1,14:02,2016-09-11T02:43:41Z,Rush,2,14,33,0,33,Shock Linwood run for no gain to the Bayl 33,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13516,400869812,2016,2,309,Louisiana,ULL,2377,McNeese,MCNS,18,14,309,Louisiana,ULL,2377,McNeese,MCNS,27,15,4,10:37,2016-09-11T09:38:27Z,Rush,3,1,99,0,99,Elijah McGuire run for no gain to the McNSt 1,109.0,13.0,23.6,15.7,26.4,59.5,178.2,0.9,44.9,179.4,4.0,1.6,71.3,357.6,5.0,7.6,10.3,7.2,58.2
13526,400869812,2016,2,309,Louisiana,ULL,2377,McNeese,MCNS,20,0,309,Louisiana,ULL,2377,McNeese,MCNS,30,15,4,3:13,2016-09-11T09:57:59Z,Pass,1,10,42,0,42,Anthony Jennings pass incomplete to Gary Haynes,109.0,13.0,23.6,15.7,26.4,59.5,178.2,0.9,44.9,179.4,4.0,1.6,71.3,357.6,5.0,7.6,10.3,7.2,58.2
13527,400869812,2016,2,309,Louisiana,ULL,2377,McNeese,MCNS,20,1,309,Louisiana,ULL,2377,McNeese,MCNS,30,15,4,3:13,2016-09-11T09:57:59Z,Rush,2,10,42,3,45,Jordan Wright run for 3 yds to the LaLaf 45,109.0,13.0,23.6,15.7,26.4,59.5,178.2,0.9,44.9,179.4,4.0,1.6,71.3,357.6,5.0,7.6,10.3,7.2,58.2
13528,400869812,2016,2,309,Louisiana,ULL,2377,McNeese,MCNS,20,2,309,Louisiana,ULL,2377,McNeese,MCNS,30,15,4,3:13,2016-09-11T09:57:59Z,Pass,3,7,45,0,45,Anthony Jennings pass incomplete to Ja'Marcus ...,109.0,13.0,23.6,15.7,26.4,59.5,178.2,0.9,44.9,179.4,4.0,1.6,71.3,357.6,5.0,7.6,10.3,7.2,58.2


In [156]:
week2_rush_feat, week2_rush_label = rush_feature_label_split(week2)
week2_pass_feat, week2_pass_label = pass_feature_label_split(week2)

In [157]:
week2_rush_pred=rush_reg.predict(week2_rush_feat)


math.sqrt(metrics.mean_squared_error(week2_rush_label, week2_rush_pred))

8.002667065657288

In [158]:
week2_pass_pred=pass_reg.predict(week2_pass_feat)


math.sqrt(metrics.mean_squared_error(week2_pass_label, week2_pass_pred))

12.226668625190213

In [159]:
pd.DataFrame(week2_pass_pred).describe()

,0
count,1761.000000
mean,7.183981
std,1.743013
min,1.401980
25%,5.999881
50%,7.206458
75%,8.399265
max,11.866324


In [160]:
pd.DataFrame(week2_pass_label).describe()

,yardsGained
count,1761.000000
mean,7.478137
std,12.306535
min,-24.000000
25%,0.000000
50%,4.000000
75%,11.000000
max,89.000000


In [161]:
import pickle
filename = 'models/rush_reg.sav'
pickle.dump(rush_reg, open(filename, 'wb'))
pickle.dump(pass_reg, open('models/pass_reg.sav', 'wb'))